## Kindle Review Sentiment Analysis Project

## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

### Best Practices
1. Preprocessing and Cleaning
2. Train Test Split
3. Apply BOW,TF-IDF or Word2Vec (Words--->Vectors)
4. Model Training
5. Model Prediction and Performance Analysis

In [13]:
import pandas as pd
data=pd.read_csv('../Kindle Reviews/all_kindle_review.csv')

In [14]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [15]:
df=data[['reviewText','rating']]

In [16]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [17]:
df.shape

(12000, 2)

In [18]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [19]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [20]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

### Step 01: Preprocessing and Cleaning

In [21]:
# As we are doing Sentiment Analysis and we have to tell that a review is positive or negative
# So we will have in rating only two values for rating less than 3, 0 means bad for rating greater than 3, 1 means good
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1) 

C:\Users\DELL\AppData\Local\Temp\ipykernel_3324\3024835294.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)


In [23]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [24]:
## 1. Lower all the cases
df['reviewText']=df['reviewText'].str.lower()

C:\Users\DELL\AppData\Local\Temp\ipykernel_3324\3052299212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


In [25]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [26]:
import re
from nltk.corpus import stopwords

In [28]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))


C:\Users\DELL\AppData\Local\Temp\ipykernel_3324\3968272687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
C:\Users\DELL\AppData\Local\Temp\ipykernel_3324\3968272687.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
C:\Users\DELL\AppData\Local\Temp\ipykernel_3324\3968272687.py:6: SettingWithCopyWarni

In [29]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [30]:
# Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [31]:
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [32]:
df['reviewText']=df['reviewText'].apply(lambda x: lemmatize_words(x))

C:\Users\DELL\AppData\Local\Temp\ipykernel_3324\3266273754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x: lemmatize_words(x))


In [33]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


### Step 02: Train Test Split

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],test_size=0.20)

In [35]:
X_test

5280     angel pawn stand alone novella introduces u as...
11527    interested reading book sex clear plotline bey...
9215     one start word one end last word could put fas...
7219     thought would instructive compare charles john...
10243    book okay 3 star writing style felt really dis...
                               ...                        
4880     love wonderful romance book thrilling end one ...
10273    usually read review purchasing book even downl...
1489     hooked soon started first page ive read availa...
4712     downloaded book right read zsadists story seri...
99       author editor better job proofing -- especiall...
Name: reviewText, Length: 2400, dtype: object

### Step:03 Words--->Vectors (BOW, TF-IDF, Word2Vec)

#### Bag Of Words (BOW)

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer=CountVectorizer()
X_train_bow=bow_vectorizer.fit_transform(X_train).toarray()
X_test_bow= bow_vectorizer.transform(X_test).toarray()

#### TF-IDF

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer()
X_train_tfidf=tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf= tfidf_vectorizer.transform(X_test).toarray()

### Word2Vec

In [41]:
from gensim.utils import simple_preprocess

In [42]:
words=[]
for sentence in X_train:
    words.append(simple_preprocess(sentence))


In [43]:
words

[['agree',
  'reviewer',
  'wrote',
  'story',
  'read',
  'like',
  'porn',
  'really',
  'think',
  'written',
  'guy',
  'dont',
  'know',
  'woman',
  'would',
  'fantasize',
  'stuff',
  'herein',
  'addition',
  'ridiculously',
  'short',
  'even',
  'proofreading',
  'errorshis',
  'hand',
  'reach',
  'caressed',
  'titty',
  'line',
  'spacing',
  'shouldnt'],
 ['im',
  'sorry',
  'dont',
  'like',
  'short',
  'story',
  'collection',
  'mascarade',
  'novel',
  'little',
  'character',
  'developement',
  'story',
  'plot',
  'save',
  'money',
  'something',
  'take',
  'longer',
  'minute',
  'read'],
 ['definitely',
  'feel',
  'mercedes',
  'like',
  'ending',
  'hate',
  'book',
  'stop',
  'real',
  'ending'],
 ['pretty',
  'nasty',
  'book',
  'honestly',
  'think',
  'orifice',
  'meant',
  'filledi',
  'came',
  'back',
  'read',
  'description',
  'see',
  'inadvertently',
  'downloaded',
  'book',
  'clearly',
  'porn',
  'case',
  'description',
  'foreshadow',
 

In [44]:
from gensim.models import Word2Vec
word2vecmodel=Word2Vec(words)

In [45]:
word2vecmodel.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'reading',
 'well',
 'series',
 'much',
 'first',
 'even',
 'short',
 'didnt',
 'know',
 'make',
 'way',
 'could',
 'sex',
 'great',
 'little',
 'two',
 'dont',
 'want',
 'thing',
 'think',
 'find',
 'romance',
 'plot',
 'also',
 'life',
 'end',
 'see',
 'im',
 'go',
 'scene',
 'enjoyed',
 'never',
 'take',
 'written',
 'woman',
 'many',
 'lot',
 'work',
 'say',
 'year',
 'kindle',
 'thought',
 'found',
 'bit',
 'interesting',
 'liked',
 'give',
 'writing',
 'novel',
 'going',
 'man',
 'got',
 'feel',
 'come',
 'loved',
 'better',
 'another',
 'enough',
 'hot',
 'back',
 'reader',
 'people',
 'still',
 'something',
 'though',
 'review',
 'star',
 'world',
 'made',
 'friend',
 'page',
 'new',
 'cant',
 'part',
 'need',
 'free',
 'keep',
 'bad',
 'wasnt',
 'doesnt',
 'relationship',
 'enjoy',
 'recommend',
 'felt',
 'together',
 'start',
 'next',
 'put',
 'best',

In [46]:
import numpy as np
def average_word2vec(doc):
    instances=[word2vecmodel.wv[word] for word in doc if word in word2vecmodel.wv.index_to_key]
    if(len(instances)<=0):
        return np.zeros(word2vecmodel.wv.vector_size,dtype=np.float32)
    return np.mean(instances,axis=0,dtype=np.float32)

In [47]:
X_train_word2vec=[]
for i in range(len(words)):
    X_train_word2vec.append(average_word2vec(words[i]))

In [49]:
len(X_train_word2vec)

9600

In [54]:
words_test=[]
for sentence in X_test:
    words_test.append(simple_preprocess(sentence))


In [55]:
X_test_word2vec=[]
for i in range(len(words_test)):
    X_test_word2vec.append(average_word2vec(words[i]))

In [56]:
len(X_test_word2vec)

2400

### Step: 04 Model Training And Performance Analysis

#### Bag Of Words (BOW)

In [60]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report
model_bow=GaussianNB()
model_bow.fit(X_train_bow,y_train)

# Prediction
y_pred_bow=model_bow.predict(X_test_bow)
print(f"Accuracy of Model Using BOW: {accuracy_score(y_test,y_pred_bow)}")
print(f"Classification of Model Using BOW: {classification_report(y_test,y_pred_bow)}")

Accuracy of Model Using BOW: 0.5845833333333333
Classification of Model Using BOW:               precision    recall  f1-score   support

           0       0.43      0.67      0.52       817
           1       0.76      0.54      0.63      1583

    accuracy                           0.58      2400
   macro avg       0.60      0.61      0.58      2400
weighted avg       0.65      0.58      0.59      2400



#### TF-IDF

In [61]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report
model_tfidf=GaussianNB()
model_tfidf.fit(X_train_tfidf,y_train)

# Prediction
y_pred_tfidf=model_tfidf.predict(X_test_tfidf)
print(f"Accuracy of Model Using TF-IDF: {accuracy_score(y_test,y_pred_tfidf)}")
print(f"Classification of Model Using TF-IDF: {classification_report(y_test,y_pred_tfidf)}")

Accuracy of Model Using TF-IDF: 0.5820833333333333
Classification of Model Using TF-IDF:               precision    recall  f1-score   support

           0       0.42      0.65      0.51       817
           1       0.75      0.55      0.63      1583

    accuracy                           0.58      2400
   macro avg       0.59      0.60      0.57      2400
weighted avg       0.64      0.58      0.59      2400



#### Word2Vec And Avg Word2Vec

In [62]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report
model_word2vec=GaussianNB()
model_word2vec.fit(X_train_word2vec,y_train)

# Prediction
y_pred_word2vec=model_word2vec.predict(X_test_word2vec)
print(f"Accuracy of Model Using word2vec: {accuracy_score(y_test,y_pred_word2vec)}")
print(f"Classification of Model Using word2vec: {classification_report(y_test,y_pred_word2vec)}")

Accuracy of Model Using word2vec: 0.5154166666666666
Classification of Model Using word2vec:               precision    recall  f1-score   support

           0       0.34      0.43      0.38       817
           1       0.66      0.56      0.60      1583

    accuracy                           0.52      2400
   macro avg       0.50      0.50      0.49      2400
weighted avg       0.55      0.52      0.53      2400

